<a href="https://colab.research.google.com/github/R0N3ldrt/Thesis/blob/main/Symbol2Symbol_P2-1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Symbol_to_Symbol Part 2

# Loading Necesary Libraries

In [ ]:
# Importing necesary libraries
# Libraries for correct code execution 

import os, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pickle
import random
import csv
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from keras.models import Sequential
from keras.layers import Dense,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping

from sklearn.mixture import GaussianMixture

import warnings
warnings.filterwarnings("ignore")

# Load Enviroment

In [ ]:
# BLOQUE PARA USAR DESDE COLAB

# Google drive loading as work station for local-usage of the files.
from google.colab import drive
drive.mount('/content/gdrive',force_remount= True)

#-----------------------------------------------------------------------------

In [ ]:
# Lista para cambiar los paths rapido.
workers = ["Ronald", "Local"]

# Cambiar el numero aqui para cambiar los paths.
worker = workers[0]

if worker == "Ronald":
  path= "/content/gdrive/MyDrive/Thesis_Workstation/ANN_dataset"
else: path = os.getcwd()

# Data Conversion

In [ ]:
def modify_data(df, train):
  # Function that converts the provided data frame into target and explicative variables.
  colnames=df.columns[1:25]
  X=df.iloc[:,1:25]
  Y=df.iloc[:,0].to_numpy().reshape(-1,1)
  if train: 
    X = sc_input.fit_transform(X)
    Y = sc_output.fit_transform(Y)
  else: 
    X = sc_input.transform(X)
    Y = sc_output.transform(Y)
  return X,Y

# Neural Network

In [ ]:
def ann():
    model = Sequential()
    model.add(Dense(12, input_dim = 24, activation = 'tanh'))
    model.add(Dense(6,activation='tanh'))
    model.add(Dense(1))
    model.compile(loss ='mean_squared_error',optimizer = 'RMSprop')
    return model

In [ ]:
def generate_and_train(X_train,Y_train):
  model_ann = KerasRegressor(build_fn=ann,epochs=500,batch_size=32)

  start_time = time.time()
  callback = EarlyStopping(monitor='loss', patience=500)
  model_ann.fit(X_train, Y_train, callbacks=[callback])
  time_train_ann = time.time() - start_time
  return model_ann

# Test Results

In [ ]:
def test_results(model,X_test,Y_test,log = True): 
  start_time = time.time()
  Y_test_pred=model.predict(X_test)
  time_eval_ann=time.time()-start_time

  real=list(list(zip(*Y_test))[0])

  #real=list(map(list, zip(*Y_train)))
  pred=list(Y_test_pred)
  print(pred)

  plt.plot(real,pred,'bo')
  x= np.linspace(0,1,100)
  plt.plot(x,x,"-r")
  plt.show()
  plt.close

  tot_abs, tot_square = 0, 0
  for i in range(len(real)):
    tot_abs += (real[i] - pred[i])
    tot_square += (real[i] - pred[i])**2
  if log:
    print('MAE:', tot_abs/len(real))
    print('MSE:', tot_square/len(real))
  return Y_test_pred

# Main

In [ ]:
global sc_input, sc_output
sc_input = MinMaxScaler()
sc_output = MinMaxScaler()


X_train,Y_train = modify_data(df_train, True)
X_test, Y_test = modify_data(df_test, False)

In [ ]:
model_ann = generate_and_train(X_train,Y_train)

pickle.dump(model_ann,open(path+"/trainedModel/new_model.pkl","wb"))

## Gradient Descent

In [ ]:
#Old path
#path_modified = path+'/modifiedData/GD/alpha1.csv'

path_modified = path+'/modifiedData/GD/GD_alpha_1.csv'
df_modified = pd.read_csv(path_modified)

In [ ]:
error_matrix_alpha_1_GD = []
for i in range(24):
  interest = 25-(i+1)
  idx1 = sum([x for x in range(interest+1,25)])*25
  idx2 = idx1 + interest*25
  points = df_modified.iloc[idx1:idx2,]
  X_mod, Y_mod = modify_data(points, False)
  aux = test_results(model_ann,X_mod,Y_mod,log = False)
  aux = sc_output.inverse_transform(aux.reshape(1,-1))[0]
  tar = sc_output.inverse_transform(Y_mod.reshape(1,-1))[0]
  mean_error = [0 for _ in range(interest)]
  for j in range(interest):
    elements = aux[j*25:j*25+25]
    targets = tar[j*25:j*25+25]
    err = 0
    for k in range(25):
      err += np.abs(elements[k]-targets[k])
    mean_error[j] = err/25
  error_matrix_alpha_1_GD.append(mean_error)

In [ ]:
og_dist = 1680
print("RESULTS FOR ORIGINAL DISTANCE:", og_dist)
a = test_results(model_ann,X_mod,Y_mod)

In [ ]:
pickle.dump(error_matrix_alpha_1_GD,open(path+"/modifiedData/GD/EM_alpha0.5.pkl","wb"))

## Z Score

In [ ]:
# Old path
#path_modified = path+'/modifiedData/Z/alpha1.csv'

path_modified = path+'/modifiedData/Z/Z_alpha_1.csv'
df_modified = pd.read_csv(path_modified)

In [ ]:
error_matrix_alpha_1_Z = []
for i in range(24):
  interest = 25-(i+1)
  idx1 = sum([x for x in range(interest+1,25)])*25
  idx2 = idx1 + interest*25
  points = df_modified.iloc[idx1:idx2,]
  X_mod, Y_mod = modify_data(points, False)
  aux = test_results(model_ann,X_mod,Y_mod,log = False)
  aux = sc_output.inverse_transform(aux.reshape(1,-1))[0]
  tar = sc_output.inverse_transform(Y_mod.reshape(1,-1))[0]
  mean_error = [0 for _ in range(interest)]
  for j in range(interest):
    elements = aux[j*25:j*25+25]
    targets = tar[j*25:j*25+25]
    err = 0
    for k in range(25):
      err += np.abs(elements[k]-targets[k])
    mean_error[j] = err/25
  error_matrix_alpha_1_Z.append(mean_error)

In [ ]:
og_dist = 1680
print("RESULTS FOR ORIGINAL DISTANCE:", og_dist)
a = test_results(model_ann,X_mod,Y_mod)

In [ ]:
pickle.dump(error_matrix_alpha_1_Z,open(path+"/modifiedData/Z/EM_alpha0.5.pkl","wb"))

# Data Analysis

In [ ]:
data_path=path+"/trainingData"
file_name="training_data.csv"
df_train=pd.read_csv(data_path+'/'+file_name)

file_name="testing_data.csv"
df_test=pd.read_csv(data_path+'/'+file_name)

print("DF train shape:",df_train.shape)
print("DF test shape:",df_test.shape)

global distances 
distances = [80*i for i in range(1,26)]

In [ ]:
%%capture
!pip install shap
import shap

# print the JS visualization code to the notebook
shap.initjs()

## Plots

In [ ]:
def var_boxplot(df):
  total_items = len(df.columns)
  items_per_row = 4
  total_rows = math.ceil(total_items / items_per_row)


  fig = make_subplots(rows=total_rows, cols=items_per_row)

  cur_row = 1
  cur_col = 1

  for index, column in enumerate(df.columns):
      fig.add_trace(go.Box(y=df[column], name=column), row=cur_row, col=cur_col)
      
      if cur_col % items_per_row == 0:
          cur_col = 1
          cur_row = cur_row + 1
      else:
          cur_col = cur_col + 1
      fig.update_layout(height=1000, width=750,  showlegend=False)
  fig.show() 

In [ ]:
def line_plots_and_params(df,pol_deg,plot = True):
  total_items = len(df.columns)
  items_per_row = 3
  total_rows = math.ceil(total_items / items_per_row)

  fig = make_subplots(rows=total_rows, cols=items_per_row, subplot_titles=df.columns)

  cur_row = 1
  cur_col = 1

  parameters = {}

  for index, column in enumerate(df.columns):
      fig.add_trace(go.Scattergl(y=df[column], 
                              x=df['dist'], 
                              mode="markers", 
                              marker=dict(size=3)), 
                    row=cur_row, 
                    col=cur_col)
      
      fit = np.polyfit(df['dist'], df[column],pol_deg)
      poly = np.poly1d(fit)
      intercept = poly(np.unique(df_train['dist']))
      parameters[column] = poly
      
      fig.add_trace(go.Scatter(x=np.unique(df["dist"]), 
                              y=intercept, 
                              line=dict(color='red', width=1)), 
                    row=cur_row, 
                    col=cur_col)
      
      if cur_col % items_per_row == 0:
          cur_col = 1
          cur_row = cur_row + 1
      else:
          cur_col = cur_col + 1
      fig.update_layout(height=2500, width=1850, showlegend=False)
  if plot: fig.show()
  return parameters

In [ ]:
params = line_plots_and_params(df_train,5,plot=True)

## Model Analysis

In [ ]:
names = df_test.columns[1:]
explainer = shap.KernelExplainer(model_ann.model, X_train[:100])
shap_values = explainer.shap_values(X_test[:100])
shap.summary_plot(shap_values, X_test, plot_type='bar',feature_names=names )